In [1]:
import os
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"

In [2]:
%matplotlib nbagg
%load_ext line_profiler

In [3]:
import numpy as np
import distributed as dd

from libertem import api
from libertem.udf.stddev import StdDevUDF, merge, process_tile
from libertem.udf.sum import SumUDF
from libertem.udf.masks import ApplyMasksUDF
from libertem.executor.inline import InlineJobExecutor
from libertem.executor.dask import DaskJobExecutor, AsyncDaskJobExecutor

In [4]:
remote_client = dd.Client('localhost:31314', set_as_default=False)
local_ctx = api.Context()
remote_ctx = api.Context(executor=DaskJobExecutor(remote_client))

In [5]:
local_ds = local_ctx.load(
    'auto',
    path=r'/Users/weber/cachedata/data/Glasgow/10 um 110.blo',
    tileshape=(8,64,144)
)
remote_ds = remote_ctx.load(
    'auto',
    path=r'/Users/weber/cachedata/data/Glasgow/10 um 110.blo',
    tileshape=(8,64,144)
)

In [6]:
local_analysis = local_ctx.create_ring_analysis(dataset=local_ds)

In [7]:
local_ctx.run(local_analysis)
%time local_ctx.run(local_analysis)

CPU times: user 170 ms, sys: 4.18 ms, total: 174 ms
Wall time: 192 ms


[<AnalysisResult: intensity>]

In [8]:
remote_analysis = remote_ctx.create_ring_analysis(dataset=remote_ds)

In [9]:
remote_ctx.run(remote_analysis)
%time remote_ctx.run(remote_analysis)

CPU times: user 149 ms, sys: 12.1 ms, total: 161 ms
Wall time: 180 ms


[<AnalysisResult: intensity>]

In [10]:
mask_shape = tuple(local_ds.shape.sig)
def mask():
    return np.ones(mask_shape)

In [11]:
udf = ApplyMasksUDF(mask_factories=[mask])

In [12]:
local_ctx.run_udf(dataset=local_ds, udf=udf)
%time local_ctx.run_udf(dataset=local_ds, udf=udf)

CPU times: user 247 ms, sys: 11.7 ms, total: 259 ms
Wall time: 285 ms


{'intensity': <BufferWrapper kind=nav dtype=float64 extra_shape=(1,)>}

In [13]:
remote_ctx.run_udf(dataset=remote_ds, udf=udf)
%time remote_ctx.run_udf(dataset=remote_ds, udf=udf)

CPU times: user 215 ms, sys: 17.4 ms, total: 232 ms
Wall time: 267 ms


{'intensity': <BufferWrapper kind=nav dtype=float64 extra_shape=(1,)>}

In [14]:
udf_2 = StdDevUDF()

In [15]:
local_ctx.run_udf(dataset=local_ds, udf=udf_2)
%time local_ctx.run_udf(dataset=local_ds, udf=udf_2)

CPU times: user 127 ms, sys: 16 ms, total: 143 ms
Wall time: 168 ms


{'varsum': <BufferWrapper kind=sig dtype=float64 extra_shape=()>,
 'num_frames': <BufferWrapper kind=single dtype=object extra_shape=()>,
 'sum': <BufferWrapper kind=sig dtype=float64 extra_shape=()>}

In [16]:
remote_ctx.run_udf(dataset=remote_ds, udf=udf_2)
%time remote_ctx.run_udf(dataset=remote_ds, udf=udf_2)

CPU times: user 80 ms, sys: 7.39 ms, total: 87.4 ms
Wall time: 118 ms


{'varsum': <BufferWrapper kind=sig dtype=float64 extra_shape=()>,
 'num_frames': <BufferWrapper kind=single dtype=object extra_shape=()>,
 'sum': <BufferWrapper kind=sig dtype=float64 extra_shape=()>}